notebook used to run & test prune diffusion models

In [10]:

#? resources
#? vid 1: https://www.youtube.com/watch?v=HoKDTa5jHvg&t=177s
# explains how ddpm works
#? vid2: https://www.youtube.com/watch?v=TBCRlnwJtZU
# goes over implementation to ddpm

import random
import os
import copy
import numpy as np
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from tqdm import tqdm
from torch import optim
from helper_utils import *

from modules import UNet, UNet_conditional, EMA
import logging 
from torch.utils.tensorboard import SummaryWriter
#torch.set_default_tensor_type('torch.cuda.FloatTensor')

# library for quantization
try:
    import bitsandbytes as bnb
    print('imported bitsandbytes')
    
except:
    print('cant import bitsandbytes')
    bnb = None

logging.basicConfig(format="%(asctime)s - %(levelname)s: %(message)s", level=logging.INFO, datefmt="%I:%M:%S")

class Diffusion:
    def __init__(self, noise_steps=1000, beta_start=1e-4, beta_end=0.02, img_size=256, device="cuda", USE_GPU = True):
        self.noise_steps = noise_steps
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.img_size = img_size #resoultion of image #: side note from video --> for higher resolutions, training seperate upsamplers instead of training on bigger resolution images
        self.device = device
        
        if USE_GPU and torch.cuda.is_available():
            print("CUDAAAAAAAAAAA")
            self.use_cuda = torch.device('cuda')
        else:
            self.use_cuda = torch.device('cpu')
        
        #? right now using simple beta schedule --> open AI using cosine scheduler        
        self.beta = self.prepare_noise_schedule().to(device)
        self.alpha = 1. - self.beta
        self.alpha_hat = torch.cumprod(self.alpha, dim=0)
        
        #! try implementing cosine scheduler
        
    def prepare_noise_schedule(self):
        #? Creates a one-dimensional tensor of size steps whose values are evenly spaced from start to end, inclusive
        return torch.linspace(self.beta_start, self.beta_end, self.noise_steps)
    
    def noise_images(self, x, t):
        """Adds noise to image. You can iteratively add noise to image but vid 1 showed 
        a simplification that adds noise in 1 step. Which is this implementation
        Args:
            x (_type_): _description_
            t (_type_): _description_

        Returns:
            _type_: returns image with noise added on
        """
        sqrt_alpha_hat = torch.sqrt(self.alpha_hat[t])[:, None, None, None]
        sqrt_one_minus_alpha_hat = torch.sqrt(1 - self.alpha_hat[t])[:, None, None, None]
        E = torch.randn_like(x)
        return sqrt_alpha_hat * x + sqrt_one_minus_alpha_hat * E, E
    
    def sample_timesteps(self, n):
        """_summary_

        Args:
            n (_type_): _description_

        Returns:
            _type_: _description_
        """
        #? needed for algorithm for training
        return torch.randint(low=1, high=self.noise_steps, size=(n,))
    

    def sample(self, model, n, labels, channels=3, cfg_scale=3):
        """implements algorithm 2 from the ddpm paper in vid 1

        Args:
            model (_type_): _description_
            n (int): number of images we want to sample 

        Returns:
            _type_: _description_
        """
        logging.info(f"Sampling {n} new images....")
        #? see here for why we set model.eval() https://stackoverflow.com/questions/60018578/what-does-model-eval-do-in-pytorch
        #? essentially disables some some parts of torch for specific steps
        logging.info(f"Sampling {n} new images....")
        model.eval()
        with torch.no_grad():
            #? create initial images by sampling over normal dist (step 1)
            x = torch.randn((n, channels, self.img_size, self.img_size)).to(self.device)
            
            #? step 2, 3, 4
            for i in tqdm(reversed(range(1, self.noise_steps)), position=0):
                t = (torch.ones(n) * i).long().to(self.device) #? tensor of timestep
                predicted_noise = model(x, t, labels) #? feed that into model w/ current images
                
                #? noise
                if cfg_scale > 0:
                    uncond_predicted_noise = model(x, t, None)
                    predicted_noise = torch.lerp(uncond_predicted_noise, predicted_noise, cfg_scale)
                alpha = self.alpha[t][:, None, None, None]
                alpha_hat = self.alpha_hat[t][:, None, None, None]
                beta = self.beta[t][:, None, None, None]
                
                #? only want noise for timestemps greater than 1. done so b/c in last iteration, would make final outcome worse due to adding noise to finalized pixels
                if i > 1:
                    noise = torch.randn_like(x)
                else:
                    noise = torch.zeros_like(x)
                    
                #? alter image by removed a little bit of noise
                x = 1 / torch.sqrt(alpha) * (x - ((1 - alpha) / (torch.sqrt(1 - alpha_hat))) * predicted_noise) + torch.sqrt(beta) * noise
        
        #? switch back to train    
        model.train()
        x = (x.clamp(-1, 1) + 1) / 2 #? brings back value to 0-1 range 
        x = (x * 255).type(torch.uint8) #? bring back values to pixel range for viewing image
        return x


===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('/Users/Efran/anaconda3/envs/compsci682/lib'), WindowsPath('C')}
The following directories listed in your path were found to be non-existent: {WindowsPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {WindowsPath('module'), WindowsPath('/matplotlib_inline.backend_inline')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
DEBUG: Possible options found for libcudart.so: set()
CUDA SETUP: PyTorch settings found: CUDA_VERSION=118, Highest Compute Capability: 8.9.
CUDA SETUP: To manually override the PyTorch CUDA version please see:https://github.com/TimDettmers/bitsandbytes/blob/

### Pruning Models

In [12]:
import torch.nn.utils.prune as prune

name = 'q_linear8_mnist_ddpm_conditional_ema'

model_path = f'F:\Classes\COMPSCI 682\denoising-diffusion-pytorch-main\ddpm\models\{name}'


def check_prune(model):
  print(dict(model.named_buffers()).keys())
  print(prune.is_pruned(model))
  for name, module in model.named_modules():
      if prune.is_pruned(module) == True:
        print(prune.is_pruned(module))
          
  total_elements = 0
  total = 0
  for named_m, module in model.named_modules():
    # print(named_m)
    # if named_m in u_middle_layers:
      # print(named_m)
    for named_p, p in module.named_parameters():
      #print(f"{named_m}.{named_p}: {p.numel()}")
      total_elements += p.numel()
      total += torch.count_nonzero(p.data)

  print(total_elements)
  print(total)

def prune_model(model, prune_method, proportion):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            #print(name, module)
            prune_method(module, name='weight', amount=proportion)
            prune.remove(module, 'weight')
        elif isinstance(module, torch.nn.Linear):
            #print(name, module)
            prune_method(module, name='weight', amount=proportion)
            prune.remove(module, 'weight')
            prune_method(module, name='bias', amount=proportion)
            prune.remove(module, 'bias')
    return model


In [13]:
proportion_list = [.1, .3]
method_list = ['random', "l1_unstructured"]
prune_methods = {
    "random": prune.random_unstructured,
    "l1_unstructured": prune.l1_unstructured}


for proportion in proportion_list:
    for method in method_list:
        #? load models
        loaded_unet_model = torch.load(os.path.join(model_path, '49_ckpt.pt'))
        #loaded_unet_model.eval()
        loaded_ema_model = torch.load(os.path.join(model_path, '49_ema_ckpt.pt'))
        #loaded_ema_model.eval()
        
        check_prune(loaded_unet_model)
        check_prune(loaded_ema_model)
        print('********** BEFORE PRUNE')
        
        prune_method = prune_methods[method]
        print(proportion, method, prune_method)
        pruned_unet = prune_model(loaded_unet_model, prune_method, proportion)
        pruned_ema = prune_model(loaded_ema_model, prune_method, proportion)

        check_prune(pruned_unet)
        check_prune(pruned_ema)
        print('********** AFTER PRUNE')
        
        prune_model_path = f'models\p_{method}_{int(proportion*100)}_{name}'
        if os.path.exists(prune_model_path) == False:    
            os.makedirs(prune_model_path)
        
        torch.save(pruned_unet, os.path.join(prune_model_path, f"pruned_49_ckpt.pt"))
        torch.save(pruned_ema, os.path.join(prune_model_path, f"pruned_49_ema_ckpt.pt"))
        
        print('--------------------------------------------------------------------------------------------')


===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('/Users/Efran/anaconda3/envs/compsci682/lib'), WindowsPath('C')}
The following directories listed in your path were found to be non-existent: {WindowsPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {WindowsPath('module'), WindowsPath('/matplotlib_inline.backend_inline')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
DEBUG: Possible options found for libcudart.so: set()
CUDA SETUP: PyTorch settings found: CUDA_VERSION=118, Highest Compute Capability: 8.9.
CUDA SETUP: To manually override the PyTorch CUDA version please see:https://github.com/TimDettmers/bitsandbytes/blob/

RuntimeError: 
        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues